In [1]:
#@Imports

#############################
# region: Imports and Setup #
#############################
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload
#############################
# endregion                 #
#############################

import datetime
from datetime import datetime as dt
from datetime import timedelta as td

from concordia.associative_memory import importance_function
from concordia.components import agent as components

from examples.custom_components.setup import model_setup, measurement_setup
from examples.tpb import utils
from examples.tpb import components as tpb_components
from examples.tpb import memory as tpb_memory
from examples.tpb import plan as tpb_plan
from examples.tpb.memory import TPBAgentConfig as AgentConfig

In [2]:
SETUP_TIME=dt(year=2024,month=10,day=1,hour=18)
TIME_STEP=td(minutes=5)

model, embedder = model_setup("llama3")
measurement, clock = measurement_setup(setup_time=SETUP_TIME,time_step=TIME_STEP)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [3]:
config = AgentConfig(
  name="Alice",
  gender="female",
  context="Alice loves to cook, but doesn't know what a Space Shuttle is.",
  goal="Alice wants to become a patissier.",
  traits="Determined, scatterbrained, picky",
  extras={
    'model': model,
    'embedder': embedder,  
    'importance': importance_model.importance,
    'clock_now': clock.now
    }
)

As a young girl, Alice was determined to learn how to make the perfect chocolate chip cookie. She spent hours in the kitchen with her mother, measuring out ingredients and mixing batter until it was just right. Her scatterbrained nature meant that she often forgot where she put things, but her pickiness ensured that every cookie was a masterpiece. As she grew older, Alice's love of cooking only intensified, and she began to experiment with new recipes and ingredients.

As an adult, Alice worked as a pastry chef at a small bakery in town. Her days were filled with the sweet scent of freshly baked bread and the sound of sizzling cookies. She loved nothing more than making people happy with her creations, and her determination to be the best baker she could be drove her to

 always strive for perfection. Despite her scatterbrained tendencies, Alice was incredibly organized when it came to her work, and her pickiness ensured that every pastry that left her shop was a masterpiece.

As Alice grew older, her love of cooking never wavered, but she began to appreciate the simple things in life more and more. She loved spending time with her family and friends, sharing stories and laughter over warm meals and cups of tea. What gave her joy was seeing the smile on someone's face when they took a bite of something delicious, and what scared her was the thought of not being able to share that joy with others anymore. Despite her lack of knowledge about things like Space Shuttles (she had no idea what they were!), Alice knew that she had lived a full and happy life, filled with love, laughter, and an endless supply of cookies.When Alice was 6 years old, she helped her mother bake cookies for their family's Christmas party. She loved watching the mixture transform in

In [4]:
params = {
  "model": model,
  "config": config,
  "num_memories": 100,
  "clock_now": clock.now,
  "verbose": True,
  "num_behavs": 5,
  "num_people": 5,
  "threading": True
}

# Memory
memory = tpb_memory.EpisodicMemory(**params)

# Situation
situation = components.situation_perception.SituationPerception(
  "situation",
  model=model,
  memory=config.memory,
  agent_name=config.name,
  clock_now = clock.now,
  num_memories_to_retrieve=100,
  verbose=True
)

# TPB pipeline components
behav = tpb_components.Behaviour("behaviour", **params)
attitude = tpb_components.Attitude("attitude", components=[behav], **params)
people = tpb_components.People("people", components=[behav], **params)
motiv = tpb_components.Motivation("motivation", components=[people], **params)
norm = tpb_components.SubjectiveNorm("norm", components=[motiv], **params)
control = tpb_components.BehaviouralControl("control", components=[behav], **params)
thin_goal = tpb_components.ThinGoal("thin_goal", **params)
intention = tpb_components.BehaviouralIntention(
  "intention", 
  components=[attitude, norm, control, thin_goal], 
  **params
)
plan = tpb_plan.TPBPlan(
  "plan",
  goal = intention,
  components=[situation],
  **params
)

full_model = tpb_components.TPBModel(
  "full_model",
  components=[
    memory,
    behav,
    attitude,
    people,
    motiv,
    norm,
    control,
    thin_goal,
    situation,
    intention,
    plan
  ]
)

In [5]:
full_model.update()

[LOG] [Component] behaviour initialized.
[LOG] [Component] attitude initialized.
[LOG] [Component] people initialized.
[LOG] [Component] motivation initialized.
[LOG] [Component] control initialized.
[LOG] [Component] intention initialized.
[LOG] [Component] thin_goal initialized.
As Alice deliberated on whether to take the job offer, she recognized that accepting it would likely lead to significant lifestyle changes, including relocating and adjusting to a new work environment, while rejecting it might result in financial instability and potentially impact her professional network and future opportunities.43 years old.Memories of Alice:
[23 Jul 1986 00:00:00] When Alice was 6 years old, she helped her mother bake cookies for their family's Christmas party. She loved watching the mixture transform into golden brown treats that filled the house with a sweet aroma. As they cooled on the rack, Alice couldn't wait to taste one.
[23 Jul 1989 00:00:00] When Alice was 9 years old, she receive

In [6]:
full_model.update()

Based on Alice's traits and goals, here are five potential behaviors she can take in response to the situation:

1. **Dive into learning**: Alice can decide to learn more about Space Shuttles, maybe even watch documentaries or read articles about them. This behavior aligns with her goal of becoming a patissier, as it shows her willingness to expand her knowledge and interests.
2. **Laugh it off and move on**: Given her history of laughing at mistakes (e.g., the burnt chicken), Alice can choose to shrug off her friends' teasing and focus on enjoying the rest of the trip.
3. **Seek help from others**: As someone who is scatterbrained, Alice might benefit from seeking help from her friends or a knowledgeable person about Space Shuttles. This behavior shows that she's willing to ask for help when needed.
4. **Make a joke out of it**: Alice can try to turn the situation into a humorous one by making a lighthearted comment or joke about being clueless. This behavior reflects her playful and 

In [7]:
intention.evaluate_intentions()

array([0.13293935, 0.1368211 , 0.58157434, 0.03468246, 0.11398275])